In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import yfinance as yf
from datetime import datetime as dt, timedelta

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [ ]:
def sequential_model(company):
    # load data
    end = dt.today()
    start = end - timedelta(days=2*365)
    data = yf.download(company, start, end)

    ## PREPARE DATA

    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(data["Close"].values.reshape(-1,1))

    prediction_days = 60

    x_train = []
    y_train = []

    for x in range(prediction_days, len(scaled_data)):
        x_train.append(scaled_data[x-prediction_days:x, 0])
        y_train.append(scaled_data[x, 0])

    x_train = np.array(x_train)
    y_train = np.array(y_train)

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


    ## BUILD THE MODEL

    model = Sequential()

    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1)) # prediction of the next point

    model.compile(optimizer="adam", loss="mean_squared_error")
    model.fit(x_train, y_train, epochs=25, batch_size=32)


    ## LOAD TEST DATA

    test_start = dt.datetime(2024,1,1)
    test_end = dt.datetime(2024,5,2)#dt.datetime.now()

    test_data = yf.download(company, test_start, test_end)

    actual_prices = test_data["Close"].values

    total_dataset = pd.concat((data["Close"], test_data["Close"]))

    model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
    model_inputs = model_inputs.reshape(-1, 1)
    model_inputs = scaler.transform(model_inputs)


    ## MAKE PREDICTION ON TEST DATA
    x_test = []

    for x in range(prediction_days, len(model_inputs)):
        x_test.append(model_inputs[x-prediction_days:x, 0])

    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    predicted_prices = model.predict(x_test)
    predicted_prices = scaler.inverse_transform(predicted_prices)


    ## PREDICT NEXT DAY

    real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs+1), 0]]
    real_data = np.array(real_data)
    real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

    #print(scaler.inverse_transform(real_data[-1]))
    prediction = model.predict(real_data)
    prediction = scaler.inverse_transform(prediction)
    #print(f"Prediction {prediction}")



    return prediction >= data["Close"][-1]


In [3]:
end = dt.today()
start = end - timedelta(days=2*365)
data = yf.download("MGROS.IS", start, end)
data["Close"][-1]

[*********************100%%**********************]  1 of 1 completed


505.5

In [4]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-05-20,42.840000,42.840000,41.619999,41.799999,41.174809,1383379
2022-05-23,41.980000,42.180000,40.900002,41.599998,40.977798,1641838
2022-05-24,41.720001,41.759998,40.939999,41.020000,40.406475,2299921
2022-05-25,41.099998,41.740002,41.040001,41.599998,40.977798,1330537
2022-05-26,41.720001,43.459999,41.580002,43.279999,42.632671,3237455
...,...,...,...,...,...,...
2024-05-13,484.500000,487.500000,470.000000,473.500000,473.500000,1931974
2024-05-14,475.000000,477.750000,464.500000,466.250000,466.250000,3241812
2024-05-15,464.500000,479.000000,452.500000,475.250000,475.250000,3546908
